In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
%matplotlib inline
import albumentations as A
import os
import numpy as np
import pandas as pd

import albumentations as A
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

__print__ = print

def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

In [2]:
EPOCHS=20
SAMPLE_LEN=100
IMAGE_PATH="../input/plant-pathology-2021-fgvc8/train_images"
TRAIN_PATH="../input/plant-pathology-2021-fgvc8/train.csv"
SUB_PATH="../input/plant-pathology-2021-fgvc8/sample_submission.csv"


In [3]:
sub=pd.read_csv(SUB_PATH)
train_data=pd.read_csv(TRAIN_PATH)


In [4]:
train_data.shape

(18632, 2)

In [5]:
train_data.labels.value_counts()

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
rust                               1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [6]:
train_data.groupby("labels")

In [7]:
a=train_data['labels'].unique()
b=train_data['labels'].value_counts()

# DATASET

In [8]:
DIR_INPUT = '../input/plant-pathology-2021-fgvc8'

SEED = 42
N_FOLDS = 5
N_EPOCHS = 10
BATCH_SIZE = 64
SIZE = 224

In [9]:
train_data.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [10]:
encoded=dict(zip(list(train_data['labels'].value_counts().keys()),range(12)))
train_data=train_data.replace({'labels':encoded})

In [11]:
encoded

{'scab': 0,
 'healthy': 1,
 'frog_eye_leaf_spot': 2,
 'rust': 3,
 'complex': 4,
 'powdery_mildew': 5,
 'scab frog_eye_leaf_spot': 6,
 'scab frog_eye_leaf_spot complex': 7,
 'frog_eye_leaf_spot complex': 8,
 'rust frog_eye_leaf_spot': 9,
 'rust complex': 10,
 'powdery_mildew complex': 11}

In [12]:
train_data.head()

,image,labels
0,800113bb65efe69e.jpg,1
1,8002cb321f8bfcdf.jpg,7
2,80070f7fb5e2ccaa.jpg,0
3,80077517781fb94f.jpg,0
4,800cbf0ff87721f8.jpg,4


In [13]:
train,valid=train_test_split(train_data,stratify=train_data.labels,test_size=0.30)
#valid,_=train_test_split(valid,stratify=valid.labels,test_size=.95)


# Percentage of each labels in the validation set

In [14]:
((valid.labels.value_counts())/valid.shape[0])*100

0     25.903399
1     24.812165
2     17.066190
3      9.982111
4      8.604651
5      6.350626
6      3.685152
7      1.073345
8      0.894454
9      0.644007
10     0.518784
11     0.465116
Name: labels, dtype: float64

# Percentage of each labels in the train set

In [15]:
((train.labels.value_counts())/train.shape[0])*100

0     25.900935
1     24.819813
2     17.075602
3      9.983131
4      8.595307
5      6.356387
6      3.680417
7      1.073455
8      0.881767
9      0.644073
10     0.521392
11     0.467720
Name: labels, dtype: float64

In [16]:
train.shape

(13042, 2)

In [17]:
train_data.loc[5,'labels']

1

# Dataset

In [18]:
class PlantDataset(Dataset):
    def __init__(self,df,transforms=None):
        self.image_id=df.image.values
        self.labels=df.labels.values
        self.transforms=transforms
        
    def __len__(self):
        return len(self.image_id)
    
    def __getitem__(self,idx):
        path=DIR_INPUT+'/train_images'+'/'+self.image_id[idx]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        labels=self.labels[idx]
        
        if self.transforms:
            image=self.transforms(image=image)['image']
            
        return image,labels

In [19]:
class PlantTestDataset(Dataset):
    def __init__(self,df,transforms=None):
        self.image_id=df.image.values
        self.tranforms=transforms
        
    def __len__(self):
        return len(self.image_id)
    def __getitem__(self,idx):
        path=DIR_INPUT+'/train_images'+'/'+self.image_id[idx]
        image=cv2.imread(path,cv2.IMREAD_COLOR)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if self.transforms:
            image=self.transforms(image=image)['image']
            
        return image    
        

# Model

In [20]:
class Plantmodel(nn.Module):
    
    def __init__(self,num_classes=12):
        super().__init__()
        self.model=torchvision.models.resnet18(pretrained=True)
        in_features=self.model.fc.in_features
        self.model.fc=nn.Linear(in_features,num_classes)
    def forward(self,x):
        batch_size, C, H, W = x.shape
        x=self.model(x)
       # x=self.backbone.maxpool(x)
        
       # x=self.backbone.layer1(x)
       # x=self.backbone.layer2(x)
       # x=self.backbone.layer3(x)
       # x=self.backbone.layer4(x)
        
       # x=F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
       # x=F.dropout(x,0.25,self.training)
       # x=self.classifier(x)
        return x
        

In [21]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    # Affine
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])


In [22]:
train.head()

,image,labels
13037,db6294bdb39a00ad.jpg,2
5360,a942625fcb72e562.jpg,1
6385,aded180ce7fd4418.jpg,0
10946,cc4f857ad53138b8.jpg,1
5073,a7d5ca94958b38c5.jpg,7


In [23]:
train_dataset=PlantDataset(train,transforms_train)
train_loader=DataLoader(train_dataset,batch_size=64,num_workers=0,shuffle=True)

In [24]:
valid_dataset=PlantDataset(valid,transforms_valid)
valid_loader=DataLoader(valid_dataset,batch_size=64,num_workers=0,shuffle=False)

For each epoch we go through the following steps:
  model.train()
  iterate through each batch
 
     load image and labels
     load image and labels to device
     pass image through model and get output
     calculate loss
     calculate gradient through loss.backward()
     optimizer.step()
     zero the optimizer
 


In [25]:
device = torch.device("cuda")
model=Plantmodel()
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Plantmodel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [26]:
x=torch.tensor([[1,2,3,5],[1,2,3,5],[1,2,3,5]])
y=torch.tensor([1,2,3])
b=torch.argmax(x,dim=1)
torch.mean((y==b).type(torch.FloatTensor))

tensor(0.3333)

In [27]:

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=5e-5)
min_acc=0
for epoch in range(10):
    print(f"Epoch{epoch}")
    model.train()
    train_loss=0
    loop=tqdm(enumerate(train_loader),total=len(train_loader),leave=False)
    for step,batch in loop :
        optimizer.zero_grad()
        image=batch[0]
        labels=batch[1]
        
        image=image.to(device,dtype=torch.float)
        labels=labels.to(device,dtype=torch.long)
        
        output=model(image)
        pred=torch.exp(output)
        pred=torch.argmax(output,dim=1)
        acc=torch.mean(((labels==pred).type(torch.FloatTensor)))
       # print(f" Training Accuracy {acc*100}%")
        loss=criterion(output,labels)
        loop.set_postfix(loss=loss.item(),accuracy=acc)
       
        
       
        loss.backward()
        optimizer.step()
    model.eval()
    for data,labels in valid_loader:
        image,labels=data,labels
        image=image.to(device,dtype=torch.float)
        labels=labels.to(device,dtype=torch.long)
        
        output=model(image)
        pred=torch.exp(output)
        pred=torch.argmax(output,dim=1)
        acc=torch.mean(((labels==pred).type(torch.FloatTensor)))
        loss=criterion(output,labels)
        valid_loss=loss.item()*data.size(0)
        print(f" Validation Accuracy {acc*100}%")
        
        if min_acc< acc:
            min_acc=acc
            print(f"Validation accuracy increased {min_acc*100:.2f} --> {acc*100:.2f} \t Saving Yhe Model")
            torch.save(model.state_dict(),'saved_model.pth')

Epoch0


  0%|          | 0/204 [00:00<?, ?it/s]

 Validation Accuracy 87.5%
Validation accuracy increased 87.50 --> 87.50 	 Saving Yhe Model
 Validation Accuracy 82.8125%
 Validation Accuracy 75.0%
 Validation Accuracy 76.5625%
 Validation Accuracy 84.375%
 Validation Accuracy 76.5625%
 Validation Accuracy 87.5%
 Validation Accuracy 90.625%
Validation accuracy increased 90.62 --> 90.62 	 Saving Yhe Model
 Validation Accuracy 81.25%
 Validation Accuracy 79.6875%
 Validation Accuracy 82.8125%
 Validation Accuracy 89.0625%
 Validation Accuracy 89.0625%
 Validation Accuracy 85.9375%
 Validation Accuracy 87.5%
 Validation Accuracy 84.375%
 Validation Accuracy 76.5625%
 Validation Accuracy 76.5625%
 Validation Accuracy 93.75%
Validation accuracy increased 93.75 --> 93.75 	 Saving Yhe Model
 Validation Accuracy 90.625%
 Validation Accuracy 78.125%
 Validation Accuracy 82.8125%
 Validation Accuracy 85.9375%
 Validation Accuracy 78.125%
 Validation Accuracy 75.0%
 Validation Accuracy 79.6875%
 Validation Accuracy 90.625%
 Validation Accuracy 

  0%|          | 0/204 [00:00<?, ?it/s]

 Validation Accuracy 87.5%
 Validation Accuracy 89.0625%
 Validation Accuracy 79.6875%
 Validation Accuracy 84.375%
 Validation Accuracy 89.0625%
 Validation Accuracy 84.375%
 Validation Accuracy 89.0625%
 Validation Accuracy 95.3125%
Validation accuracy increased 95.31 --> 95.31 	 Saving Yhe Model
 Validation Accuracy 85.9375%
 Validation Accuracy 76.5625%
 Validation Accuracy 84.375%
 Validation Accuracy 90.625%
 Validation Accuracy 92.1875%
 Validation Accuracy 85.9375%
 Validation Accuracy 87.5%
 Validation Accuracy 85.9375%
 Validation Accuracy 78.125%
 Validation Accuracy 76.5625%
 Validation Accuracy 92.1875%
 Validation Accuracy 92.1875%
 Validation Accuracy 81.25%
 Validation Accuracy 85.9375%
 Validation Accuracy 85.9375%
 Validation Accuracy 81.25%
 Validation Accuracy 73.4375%
 Validation Accuracy 82.8125%
 Validation Accuracy 92.1875%
 Validation Accuracy 85.9375%
 Validation Accuracy 84.375%
 Validation Accuracy 82.8125%
 Validation Accuracy 85.9375%
 Validation Accuracy 

...

 Validation Accuracy 89.0625%
 Validation Accuracy 84.375%
 Validation Accuracy 76.5625%
 Validation Accuracy 89.0625%
 Validation Accuracy 93.75%
 Validation Accuracy 81.25%
 Validation Accuracy 92.1875%
 Validation Accuracy 93.75%
 Validation Accuracy 87.5%
 Validation Accuracy 78.125%
 Validation Accuracy 84.375%
 Validation Accuracy 92.1875%
 Validation Accuracy 90.625%
 Validation Accuracy 85.9375%
 Validation Accuracy 87.5%
 Validation Accuracy 90.625%
 Validation Accuracy 82.8125%
 Validation Accuracy 75.0%
 Validation Accuracy 90.625%
 Validation Accuracy 96.875%
 Validation Accuracy 85.9375%
 Validation Accuracy 90.625%
 Validation Accuracy 84.375%
 Validation Accuracy 82.8125%
 Validation Accuracy 78.125%
 Validation Accuracy 84.375%
 Validation Accuracy 95.3125%
 Validation Accuracy 84.375%
 Validation Accuracy 85.9375%
 Validation Accuracy 90.625%
 Validation Accuracy 89.0625%
 Validation Accuracy 85.9375%
 Validation Accuracy 89.0625%
 Validation Accuracy 82.8125%
 Validat

  0%|          | 0/204 [00:00<?, ?it/s]

 Validation Accuracy 87.5%
 Validation Accuracy 90.625%
 Validation Accuracy 79.6875%
 Validation Accuracy 92.1875%
 Validation Accuracy 85.9375%
 Validation Accuracy 84.375%
 Validation Accuracy 84.375%
 Validation Accuracy 93.75%
 Validation Accuracy 89.0625%
 Validation Accuracy 76.5625%
 Validation Accuracy 82.8125%
 Validation Accuracy 92.1875%
 Validation Accuracy 90.625%
 Validation Accuracy 89.0625%
 Validation Accuracy 92.1875%
 Validation Accuracy 90.625%
 Validation Accuracy 90.625%
 Validation Accuracy 84.375%
 Validation Accuracy 92.1875%
 Validation Accuracy 96.875%
 Validation Accuracy 87.5%
 Validation Accuracy 90.625%
 Validation Accuracy 87.5%
 Validation Accuracy 82.8125%
 Validation Accuracy 75.0%
 Validation Accuracy 82.8125%
 Validation Accuracy 96.875%
 Validation Accuracy 84.375%
 Validation Accuracy 84.375%
 Validation Accuracy 84.375%
 Validation Accuracy 87.5%
 Validation Accuracy 84.375%
 Validation Accuracy 84.375%
 Validation Accuracy 89.0625%
 Validation 

  0%|          | 0/204 [00:00<?, ?it/s]

 Validation Accuracy 89.0625%
 Validation Accuracy 89.0625%
 Validation Accuracy 81.25%
 Validation Accuracy 89.0625%
 Validation Accuracy 92.1875%
 Validation Accuracy 81.25%
 Validation Accuracy 90.625%
 Validation Accuracy 92.1875%
 Validation Accuracy 90.625%
 Validation Accuracy 78.125%
 Validation Accuracy 79.6875%
 Validation Accuracy 95.3125%
 Validation Accuracy 92.1875%
 Validation Accuracy 89.0625%
 Validation Accuracy 89.0625%
 Validation Accuracy 90.625%
 Validation Accuracy 82.8125%
 Validation Accuracy 78.125%
 Validation Accuracy 92.1875%
 Validation Accuracy 96.875%
 Validation Accuracy 87.5%
 Validation Accuracy 90.625%
 Validation Accuracy 87.5%
 Validation Accuracy 84.375%
 Validation Accuracy 76.5625%
 Validation Accuracy 81.25%
 Validation Accuracy 92.1875%
 Validation Accuracy 85.9375%
 Validation Accuracy 84.375%
 Validation Accuracy 85.9375%
 Validation Accuracy 89.0625%
 Validation Accuracy 85.9375%
 Validation Accuracy 89.0625%
 Validation Accuracy 89.0625%
 

In [28]:
torch.save(model.state_dict(),'Plant_2021_epoch10.pth')